In [10]:
# Input Data within this string
data = """
Green, Green, 2021-03-24T07:58:30 
Green, Red, 2021-03-24T09:58:30 
Red, Red, 2021-03-25T11:58:30
"""

In [11]:
import datetime
import dateutil.parser

# Input Data (for testing it locally with a CSV file)
# path = input("Enter the complete path to the input CSV file : ")
# file = open(path)
# data = file.read()
rides_array_unsorted = data.split("\n")

# Sort Rides by Dates (using Binary Tree)
class Node:
   def __init__(self, data):
      self.left = None
      self.right = None
      self.data = data
   def insert(self, data):
      if self.data:
         if data[2] < self.data[2]:
            if self.left is None:
               self.left = Node(data)
            else:
               self.left.insert(data)
         else:
            if self.right is None:
               self.right = Node(data)
            else:
               self.right.insert(data)
      else:
         self.data = data
   def inorderTraversal(self, root):
      res = []
      if root:
         res = self.inorderTraversal(root.left)
         res.append(root.data)
         res = res + self.inorderTraversal(root.right)
      return res

root_node_set = False
for i in range(0, len(rides_array_unsorted)):
    if len(rides_array_unsorted[i])<2: # Blank Row
        continue
    if root_node_set == False:
        root = Node(rides_array_unsorted[i].split(","))
        root_node_set = True
    else:
        root.insert(rides_array_unsorted[i].split(","))
rides_array = root.inorderTraversal(root)

# Main Logic (Calculate Fare)
result_fare = 0
daily_fare_cap_cache = [dateutil.parser.parse(rides_array[0][2]), {"GreenGreen": 0, "RedRed": 0, "GreenRed": 0, "RedGreen": 0}]
weekly_fare_cap_cache = [dateutil.parser.parse(rides_array[0][2]), {"GreenGreen": 0, "RedRed": 0, "GreenRed": 0, "RedGreen": 0}]
for ride in rides_array:
    
    # Setting Standard vars
    this_date = dateutil.parser.parse(ride[2])
    this_time = [this_date.hour, this_date.minute, this_date.second]
    this_day = this_date.weekday()
    ride[0] = ride[0].strip()
    ride[1] = ride[1].strip()

    # Peak Hours
    is_peak = False
    if this_day < 5:    # Weekdays
        if this_time[0] in [8, 9, 17, 18]:
            is_peak = True
        elif this_time[0] == 16 and this_time[1]>29:
            is_peak = True
    elif this_day == 5:  # Saturday
        if 9 < this_time[0] < 14 or 17 < this_time[0] < 23:
            is_peak = True
    elif this_day == 6:  # Sunday
        if 17 < this_time[0] < 23:
            is_peak = True

    # Fare of the current ride
    this_fare = 0
    if ride[0]=="Green" and ride[1]=="Green":
        this_fare = 2 if is_peak else 1
    if ride[0]=="Red" and ride[1]=="Red":
        this_fare = 3 if is_peak else 2
    if ride[0]=="Green" and ride[1]=="Red":
        this_fare = 4 if is_peak else 3
    if ride[0]=="Red" and ride[1]=="Green":
        this_fare = 3 if is_peak else 2

    # Daily Fare Cap
    day_cap_fare = 99   # Set at a random high number since the lowest fare is needed later (calculated ahead)
    if this_date == daily_fare_cap_cache[0]:
        existing_fare = daily_fare_cap_cache[1][ride[0]+ride[1]]
        if ride[0] == "Green" and ride[1] == "Green":
            if existing_fare+this_fare > 8:
                day_cap_fare = 0 if existing_fare == 8 else 8-existing_fare
        if ride[0] == "Red" and ride[1] == "Red":
            if existing_fare+this_fare > 12:
                day_cap_fare = 0 if existing_fare == 12 else 12-existing_fare
        if ride[0] == "Green" and ride[1] == "Red":
            if existing_fare+this_fare > 15:
                day_cap_fare = 0 if existing_fare == 15 else 15-existing_fare
        if ride[0] == "Red" and ride[1] == "Green":
            if existing_fare+this_fare > 15:
                day_cap_fare = 0 if existing_fare == 15 else 15-existing_fare
    else:
        daily_fare_cap_cache = [dateutil.parser.parse(ride[2]), {"GreenGreen": 0, "RedRed": 0, "GreenRed": 0, "RedGreen": 0}]

    # Weekly Fare Cap
    week_cap_fare = 99  # Set at a random high number since the lowest fare is needed later (calculated ahead)
    is_same_week = False
    day_diff = this_date - weekly_fare_cap_cache[0]
    if day_diff.days < 7:
        if this_date.weekday() >= weekly_fare_cap_cache[0].weekday():
            is_same_week = True
    if is_same_week:
        existing_fare = weekly_fare_cap_cache[1][ride[0]+ride[1]]
        if ride[0] == "Green" and ride[1] == "Green":
            if existing_fare+this_fare > 55:
                week_cap_fare = 0 if existing_fare == 55 else 55-existing_fare
        if ride[0] == "Red" and ride[1] == "Red":
            if existing_fare+this_fare > 70:
                week_cap_fare = 0 if existing_fare == 70 else 70-existing_fare
        if ride[0] == "Green" and ride[1] == "Red":
            if existing_fare+this_fare > 90:
                week_cap_fare = 0 if existing_fare == 90 else 90-existing_fare
        if ride[0] == "Red" and ride[1] == "Green":
            if existing_fare+this_fare > 90:
                week_cap_fare = 0 if existing_fare == 90 else 90-existing_fare
    else:
        weekly_fare_cap_cache = [dateutil.parser.parse(ride[2]), {"GreenGreen": 0, "RedRed": 0, "GreenRed": 0, "RedGreen": 0}]

    # Lowest Fare (select the lowest fare out of regular fare, day cap fare or week cap fare)
    final_fare_1 = this_fare if this_fare < day_cap_fare else day_cap_fare
    final_fare = final_fare_1 if final_fare_1 < week_cap_fare else week_cap_fare

    # Update Fare to cache and final count
    result_fare += final_fare
    daily_fare_cap_cache[1][ride[0] + ride[1]] += final_fare
    weekly_fare_cap_cache[1][ride[0] + ride[1]] += final_fare

# Return Result
print("$" + str(result_fare))

$7
